# Sinus CT Data Exploration
## Initial analysis of CT volume characteristics and preparation for ML modeling

In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from pathlib import Path
import json
from scipy import ndimage
import pandas as pd

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## Load CT Volume and Metadata

In [ ]:
# Load the processed NIfTI
nifti_path = Path('../data/processed/sinus_ct.nii.gz')
img = nib.load(str(nifti_path))
volume = img.get_fdata().astype(np.float32)
spacing = img.header.get_zooms()[:3]

# Load metadata
with open('../docs/last_run_meta.json') as f:
    meta = json.load(f)

print(f"Volume shape: {volume.shape}")
print(f"Voxel spacing: {spacing} mm")
print(f"Physical dimensions: {[s*d for s,d in zip(volume.shape, spacing)]} mm")
print(f"Study date: {meta['study_date']}")
print(f"HU range: [{volume.min():.1f}, {volume.max():.1f}]")

## Intensity Distribution Analysis
Understanding HU distribution is critical for segmentation thresholds

In [ ]:
# Create histogram of HU values
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Full range histogram
axes[0].hist(volume.flatten(), bins=100, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Hounsfield Units')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Full HU Distribution')
axes[0].axvline(-1000, color='red', linestyle='--', label='Air', alpha=0.7)
axes[0].axvline(0, color='green', linestyle='--', label='Water/Soft tissue', alpha=0.7)
axes[0].legend()

# Focus on sinus-relevant range [-1000, 400]
sinus_range = volume[(volume >= -1000) & (volume <= 400)]
axes[1].hist(sinus_range.flatten(), bins=100, alpha=0.7, color='coral', edgecolor='black')
axes[1].set_xlabel('Hounsfield Units')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Sinus-Relevant Range [-1000, 400] HU')
axes[1].axvline(-400, color='purple', linestyle='--', label='Air cavity threshold', alpha=0.7)
axes[1].legend()

plt.tight_layout()
plt.show()

# Calculate key statistics
print(f"\nKey Statistics:")
print(f"  Mean HU: {volume.mean():.1f}")
print(f"  Std HU: {volume.std():.1f}")
print(f"  Air voxels (< -400 HU): {(volume < -400).sum() / volume.size * 100:.2f}%")
print(f"  Soft tissue voxels (-100 to 100 HU): {((volume > -100) & (volume < 100)).sum() / volume.size * 100:.2f}%")

## Anatomical Slice Visualization
Examine axial, sagittal, and coronal views

In [ ]:
# Select middle slices
mid_axial = volume.shape[0] // 2
mid_sagittal = volume.shape[1] // 2
mid_coronal = volume.shape[2] // 2

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Axial (head-to-feet)
axes[0].imshow(volume[mid_axial, :, :], cmap='gray', vmin=-1000, vmax=400)
axes[0].set_title(f'Axial Slice {mid_axial}')
axes[0].axis('off')

# Sagittal (left-to-right)
axes[1].imshow(volume[:, mid_sagittal, :], cmap='gray', vmin=-1000, vmax=400)
axes[1].set_title(f'Sagittal Slice {mid_sagittal}')
axes[1].axis('off')

# Coronal (front-to-back)
axes[2].imshow(volume[:, :, mid_coronal], cmap='gray', vmin=-1000, vmax=400)
axes[2].set_title(f'Coronal Slice {mid_coronal}')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## Air Cavity Segmentation (Baseline)
Simple threshold-based segmentation to identify nasal/sinus air spaces

In [ ]:
# Threshold for air cavities
air_threshold = -400  # HU
air_mask = volume < air_threshold

# Remove small noise with morphological operations
from scipy.ndimage import binary_opening, binary_closing
air_mask_clean = binary_opening(air_mask, structure=np.ones((3, 3, 3)))
air_mask_clean = binary_closing(air_mask_clean, structure=np.ones((5, 5, 5)))

# Calculate air volume
voxel_volume_mm3 = np.prod(spacing)
air_volume_mm3 = air_mask_clean.sum() * voxel_volume_mm3
air_volume_ml = air_volume_mm3 / 1000

print(f"Detected air cavity volume: {air_volume_ml:.2f} mL")
print(f"Air cavity voxels: {air_mask_clean.sum()}")
print(f"Air fraction: {air_mask_clean.sum() / volume.size * 100:.2f}%")

In [ ]:
# Visualize air segmentation overlay
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

slices = [mid_axial - 20, mid_axial, mid_axial + 20]
for i, slice_idx in enumerate(slices):
    # Original
    axes[0, i].imshow(volume[slice_idx, :, :], cmap='gray', vmin=-1000, vmax=400)
    axes[0, i].set_title(f'Original - Slice {slice_idx}')
    axes[0, i].axis('off')
    
    # Overlay air mask
    axes[1, i].imshow(volume[slice_idx, :, :], cmap='gray', vmin=-1000, vmax=400)
    axes[1, i].imshow(air_mask_clean[slice_idx, :, :], cmap='Reds', alpha=0.4)
    axes[1, i].set_title(f'Air Segmentation - Slice {slice_idx}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## Region of Interest (ROI) Analysis
Identify sinus regions for focused training

In [ ]:
from scipy.ndimage import label

# Label connected components
labeled_air, num_features = label(air_mask_clean)
print(f"Found {num_features} distinct air cavities")

# Analyze each component
component_stats = []
for i in range(1, num_features + 1):
    component = (labeled_air == i)
    volume_ml = component.sum() * voxel_volume_mm3 / 1000
    
    # Get bounding box
    coords = np.where(component)
    bbox = [
        (coords[0].min(), coords[0].max()),
        (coords[1].min(), coords[1].max()),
        (coords[2].min(), coords[2].max())
    ]
    
    component_stats.append({
        'component': i,
        'volume_ml': volume_ml,
        'voxels': component.sum(),
        'bbox': bbox
    })

# Sort by volume
component_stats.sort(key=lambda x: x['volume_ml'], reverse=True)

# Display top 10 largest cavities
df = pd.DataFrame(component_stats[:10])
print("\nTop 10 Largest Air Cavities:")
print(df[['component', 'volume_ml', 'voxels']].to_string(index=False))

## Prepare Training Patches
Extract 96³ patches for MONAI training

In [ ]:
# Define patch extraction function
def extract_patches(volume, mask, patch_size=(96, 96, 96), num_patches=10, foreground_ratio=0.3):
    """
    Extract patches with at least foreground_ratio of mask voxels.
    """
    patches = []
    mask_patches = []
    
    # Find valid centers (not too close to edges)
    valid_centers = []
    for i in range(patch_size[0]//2, volume.shape[0] - patch_size[0]//2, 10):
        for j in range(patch_size[1]//2, volume.shape[1] - patch_size[1]//2, 10):
            for k in range(patch_size[2]//2, volume.shape[2] - patch_size[2]//2, 10):
                # Check if patch has sufficient foreground
                patch_mask = mask[
                    i-patch_size[0]//2:i+patch_size[0]//2,
                    j-patch_size[1]//2:j+patch_size[1]//2,
                    k-patch_size[2]//2:k+patch_size[2]//2
                ]
                if patch_mask.mean() >= foreground_ratio:
                    valid_centers.append((i, j, k))
    
    # Sample random patches
    if len(valid_centers) > num_patches:
        selected = np.random.choice(len(valid_centers), num_patches, replace=False)
        valid_centers = [valid_centers[i] for i in selected]
    
    for i, j, k in valid_centers:
        patch = volume[
            i-patch_size[0]//2:i+patch_size[0]//2,
            j-patch_size[1]//2:j+patch_size[1]//2,
            k-patch_size[2]//2:k+patch_size[2]//2
        ]
        patch_mask = mask[
            i-patch_size[0]//2:i+patch_size[0]//2,
            j-patch_size[1]//2:j+patch_size[1]//2,
            k-patch_size[2]//2:k+patch_size[2]//2
        ]
        patches.append(patch)
        mask_patches.append(patch_mask)
    
    return patches, mask_patches

# Extract sample patches
patches, mask_patches = extract_patches(volume, air_mask_clean, num_patches=5, foreground_ratio=0.2)
print(f"Extracted {len(patches)} training patches")

In [ ]:
# Visualize patches
fig, axes = plt.subplots(len(patches), 3, figsize=(12, 4*len(patches)))
if len(patches) == 1:
    axes = axes.reshape(1, -1)

for i, (patch, mask_patch) in enumerate(zip(patches, mask_patches)):
    mid = patch.shape[0] // 2
    
    axes[i, 0].imshow(patch[mid, :, :], cmap='gray', vmin=-1000, vmax=400)
    axes[i, 0].set_title(f'Patch {i+1} - Image')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask_patch[mid, :, :], cmap='Reds')
    axes[i, 1].set_title(f'Patch {i+1} - Mask')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(patch[mid, :, :], cmap='gray', vmin=-1000, vmax=400)
    axes[i, 2].imshow(mask_patch[mid, :, :], cmap='Reds', alpha=0.4)
    axes[i, 2].set_title(f'Patch {i+1} - Overlay')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

## Next Steps
1. Generate synthetic training data with controlled inflammation patterns
2. Train MONAI 3D U-Net on augmented dataset
3. Extract PyRadiomics features for longitudinal tracking
4. Build comparative analysis against literature values